# Урок 8. Снижение размерности данных

### 1. Можно ли отобрать наиболее значимые признаки с помощью PCA?

Да, можно. 

Т.к смысл метода заключается в том, что с каждой главной компонентой связана определённая доля общей дисперсии исходного набора данных, а дисперсия, является мерой изменчивости данных, то она может отражать уровень информативности этих данных.

Вдоль некоторых осей исходного пространства признаков изменчивость может быть большой, вдоль других — малой, а вдоль третьих вообще отсутствовать. Чем меньше дисперсия данных вдоль оси, тем менее значим вклад переменной, связанной с данной осью.

Следовательно, решая задачу метода главных компонент мы строим новое пространство признаков меньшей размерности. Максимизируя дисперсию мы вактически находим наиболее важные признаки, а посчитав долю дисперсии мы поймем какие признаки наиболее значимые.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

### Загрузим датасет из урока и проведем предварительные преобразования

In [2]:
iris = datasets.load_iris()
X = iris.data

# print(X)

# Для начала отмасштабируем выборку
X_ = X.astype(float)

rows, cols = X_.shape

# центрирование - вычитание из каждого значения среднего по строке
means = X_.mean(0)
for i in range(rows):
    for j in range(cols):
        X_[i, j] -= means[j]

# деление каждого значения на стандартное отклонение
std = np.std(X_, axis=0)
for i in range(cols):
    for j in range(rows):
        X_[j][i] /= std[i]

# Найдем собственные векторы и собственные значения
 
covariance_matrix = X_.T.dot(X_)

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
# eig_pairs.sort(key=lambda x: x[0], reverse=True)

print('Собственные значения в порядке убывания:')
for i in eig_pairs:
    print(i[0], i[1])

Собственные значения в порядке убывания:
437.77467247979916 [ 0.52106591 -0.26934744  0.5804131   0.56485654]
137.1045707202104 [-0.37741762 -0.92329566 -0.02449161 -0.06694199]
22.013531335697223 [-0.71956635  0.24438178  0.14212637  0.63427274]
3.1072254642928456 [ 0.26128628 -0.12350962 -0.80144925  0.52359713]


### Посчитаем долю дисперсии и кумулятивную долю дисперсии

In [3]:
eig_sum = sum(eig_values)
var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описвыаемая каждой из компонент \n{var_exp}')

# а теперя оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описвыаемая каждой из компонент 
[72.9624454132999, 22.85076178670175, 3.6689218892828723, 0.5178709107154745]
Кумулятивная доля дисперсии по компонентам 
[ 72.96244541  95.8132072   99.48212909 100.        ]


Таким образом мы получили долю влияния признаков и можем из отбирать по принципу, чем больше доля, тем пважнее призанк.

Из примера видим, что первые 3 принзнака описывают 99% данных, а например, первые 2 описывают почти 96% данных. Таким образом можем взять либо первые 2, либо первые 3 в зависимоти от бизнес-требований